# Задание по курсу «Дискретная оптимизация», МФТИ, весна 2017

## Задача 3-1. Задача TSP: инкрементальные алгоритмы.

В этой задаче Вам предлагается сравнить алгоритмы Nearest Neighbour и Nearest Insertion в задаче Euclidean TSP.

**Даны:**
* Координаты точек плоскости, являющихся вершинами графа.

**Найти:**
* Перестановку вершин, задающих минимальный по длине гамильтонов цикл в графе.

Сделайте следующее:
* Скачайте файл [`tsp-instances.zip`](https://github.com/dainiak/discrete-optimization-course/raw/master/tsp-instances.zip) и разархивируйте из него файлы со входами задачи TSP.
* Реализуйте функции `solve_tsp_nearest_neighbour` и `solve_tsp_nearest_insertion`.
* Запустите функцию `run_all()`, чтобы протестировать свой код и сравнить качество решений, получаемых Nearest Neighbour и Nearest Insertion. Сильно ли они отличаются? Запишите свои качественные выводы в 1-2 предложениях в последней ячейке ipynb-файла.

In [1]:
def read_tsp_instance(filename):
    with open(filename, 'r') as file:
        coordinates = []
        for line in file:
            line = line.strip().lower()
            if line.startswith('dimension'):
                coordinates = [(0,0)] * int(line.split()[-1])
            tokens = line.split()
            if len(tokens) == 3 and tokens[0].isdecimal():
                tokens = line.split()
                coordinates[int(tokens[0])-1] = tuple(map(float, tokens[1:]))
        return coordinates

from math import sqrt

def euclidean_distance(point1, point2):
    return sqrt((point1[0]-point2[0]) ** 2 + (point1[1]-point2[1]) ** 2)
    
def calculate_tour_length(instance, permutation):
    n = len(permutation)
    return sum(euclidean_distance(instance[permutation[i]], instance[permutation[(i+1) % n]]) for i in range(len(permutation)))

In [2]:
import random

def solve_tsp_nearest_neighbour(instance):
    permutation = []
    not_used = set(range(len(instance)))
    
    new_vertex = random.choice(tuple(not_used))
    permutation.append(new_vertex)
    not_used.remove(new_vertex)
    
    for i in range(len(instance) - 1):
        new_vertex = -1
        min_distance = -1
        for v in not_used:
            new_distance = euclidean_distance(instance[permutation[i]],
                                              instance[v])
            if new_distance < min_distance or min_distance == -1:
                new_vertex = v
                min_distance = new_distance
        permutation.append(new_vertex)
        not_used.remove(new_vertex)
    
    return permutation

In [3]:
def replacement_loss(new, u, v):
    return euclidean_distance(u, new) + \
           euclidean_distance(v, new) - \
           euclidean_distance(u, v)

def solve_tsp_nearest_insertion(instance):
    permutation = []
    not_used = set(range(len(instance)))
    
    begin, end = -1, -1
    min_distance = -1
    for v in range(len(instance)):
        for u in range(v):
            new_distance = euclidean_distance(instance[u],
                                              instance[v])
            if new_distance < min_distance or min_distance == -1:
                begin = u
                end = v
                min_distance = new_distance       
    permutation.extend([begin, end])
    not_used.remove(begin)
    not_used.remove(end)
    
    for i in range(len(instance) - 2):
        new_vertex = -1
        min_distance = -1
        
        for u in not_used:
            vertex_distance = -1
            for v in permutation:
                new_distance = euclidean_distance(instance[u], instance[v])
                if new_distance < vertex_distance or vertex_distance == -1:
                    vertex_distance = new_distance
            if vertex_distance < min_distance or min_distance == -1:
                new_vertex = u
        not_used.remove(new_vertex)
        
        split_place = -1
        min_distance = -1
        for j in range(i + 2):
            new_distance = replacement_loss(instance[new_vertex], instance[permutation[j]],
                                            instance[permutation[(j + 1) % (i + 2)]])
            if new_distance < min_distance or min_distance == -1:
                split_place = j
                min_distance = new_distance
        
        permutation = permutation[0:split_place + 1] + \
                      [new_vertex] + permutation[split_place + 1:i + 2]
    return permutation

In [4]:
import time
from os.path import exists

def run_all():
    instance_filenames = ['d198.tsp', 'd493.tsp', 'd657.tsp', 'd2103.tsp', 'pr107.tsp', 'pr152.tsp', 'pr439.tsp']
    for filename in instance_filenames:
        if not exists(filename):
            print('File not found: “{}”. Skipping this instance.'.format(filename))
            continue
        instance = read_tsp_instance(filename)
        print('Solving instance {}…'.format(filename), end='')
        time_start = time.monotonic()
        quality_nn = calculate_tour_length(instance, solve_tsp_nearest_neighbour(instance))
        time_nn = time.monotonic()-time_start
        time_start = time.monotonic()
        quality_ni = calculate_tour_length(instance, solve_tsp_nearest_insertion(instance))
        time_ni = time.monotonic()-time_start
        print(' done in {:.2} seconds with tour length {} using NN and in {:.2} seconds with tour length {} using NI'.format(time_nn, int(quality_nn), time_ni, int(quality_ni)))

In [5]:
run_all()

Solving instance d198.tsp… done in 0.03 seconds with tour length 19376 using NN and in 1.1 seconds with tour length 17684 using NI
Solving instance d493.tsp… done in 0.098 seconds with tour length 44302 using NN and in 1.6e+01 seconds with tour length 42286 using NI
Solving instance d657.tsp… done in 0.18 seconds with tour length 63376 using NN and in 3.9e+01 seconds with tour length 59132 using NI
Solving instance d2103.tsp… done in 1.8 seconds with tour length 88554 using NN and in 1.3e+03 seconds with tour length 94106 using NI
Solving instance pr107.tsp… done in 0.0045 seconds with tour length 49128 using NN and in 0.17 seconds with tour length 46397 using NI
Solving instance pr152.tsp… done in 0.0092 seconds with tour length 88767 using NN and in 0.49 seconds with tour length 81002 using NI
Solving instance pr439.tsp… done in 0.081 seconds with tour length 136200 using NN and in 9.7 seconds with tour length 134165 using NI


## Выводы
Алгоритм NI часто дает результат лучше, чем NN, но во всех данных случаях различие было не более чем на 10%. Более того, NI не всегда выигрывает у NN в плане результата, а в плане асимптотики и, соответственно, времени работы, он значительно хуже.